In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import submitit
import sys
from os.path import join

sys.path.append("../../src")
import simulate
import pandas as pd
from tqdm import tqdm

/u/project/pasaniuc/kangchen/2021-h2gene/experiments/01_simulate


In [2]:
CHROM = 1
SUMSTATS_DIR = "/u/project/pasaniuc/pasaniucdata/UKBB_IMPUTED_LD_SUMSTATS"
PLINK_PATH = join(SUMSTATS_DIR, f"genotype/raw/chr{CHROM}")
LD_PATH = join(SUMSTATS_DIR, "ld/")
LDSCORE_PATH = join(SUMSTATS_DIR, f"ldscore/chr{CHROM}.l2.ldscore.gz")
GENE_LIST_PATH = "../../data/gene_list.bed"
LD_BLOCK_PATH = "../../data/ld_blocks.bed"

In [12]:
bim = pd.read_csv(
    PLINK_PATH + ".bim",
    header=None,
    delim_whitespace=True,
    names=["CHR", "SNP", "CM", "BP", "A1", "A2"],
)
mean_std = pd.read_csv(PLINK_PATH + "_mean_std.txt", delim_whitespace=True)
ldscore = pd.read_csv(LDSCORE_PATH, delim_whitespace=True)

bim["MAF"] = 1.0 - mean_std["mean"].values / 2
bim["LDSCORE"] = ldscore["L2"]

bim.to_csv("data/snp_info.tsv", sep="\t", index=False)

df_gene = pd.read_csv(
    GENE_LIST_PATH,
    header=None,
    names=["CHR", "START", "STOP", "STRAND", "NAME"],
    delim_whitespace=True,
)
df_gene = df_gene[df_gene.CHR == str(CHROM)].reset_index(drop=True)

df_partition = pd.read_csv(LD_BLOCK_PATH, delim_whitespace=True)
df_partition.columns = ["CHR", "START", "STOP"]
df_partition["CHR"] = df_partition["CHR"].apply(lambda x: int(x[3:]))
df_partition = df_partition[df_partition.CHR == CHROM]
df_partition.to_csv("data/partition.bed", sep="\t", index=False)

# Here in simulation, 

In [36]:
MIN_GENE_N_SNP = 25

# for each gene, filter out genes overlapping with LD block boundary / no SNPs
filtered_index = []
for par_i, par in tqdm(df_partition.iterrows(), total=df_partition.shape[0]):
    df_par_gene = df_gene.loc[
        (par.START <= df_gene.START) & (df_gene.STOP < par.STOP), :
    ]
    df_par_bim = bim.loc[(par.START <= bim.BP) & (bim.BP < par.STOP), :]
    for gene_i, gene in df_par_gene.iterrows():
        if (
            sum((gene.START <= df_par_bim.BP) & (df_par_bim.BP < gene.STOP))
            > MIN_GENE_N_SNP
        ):
            filtered_index.append(gene_i)

df_gene = df_gene.iloc[filtered_index, :].reset_index(drop=True)
df_gene.to_csv("data/df_gene.tsv", sep="\t", index=False)

100%|██████████| 133/133 [00:01<00:00, 85.99it/s]
